# Estimating Contamination Rate with hapCon for Male aDNA Samples

Starting from version 0.4a1, hapROH package now has an extension called hapCon to estimate contamination for male aDNA samples.

This small notebook walks you through how to use hapCon to run estimate contamination in your male aDNA sample. We will use one 1240k sample SUA001, from Sardinia, and a WGS sample DA43, from Mongolia, XiongNu to illustrate hapCon's usage on two different reference panels. In this tutorial, both samples have been downsampled to 0.1x to keep runtime minimal.

You can download the two test BAM files and the reference panel from [this link](https://zenodo.org/record/6619138#.Yr9CzuzP1jc).


# hapCon with 1240k Reference Panel

hapCon takes as input either a BAM file, output from samtool's mpileup module or output from [BamTable](https://bioinf.eva.mpg.de/BamTable/), a BAM file utility developed by Kay Prüfer and Michael Siebauer. Running hapCon with directly from BAM file is significantly slower than the other two options. <span style='color:red'>We recommend using BamTable as your first choice as it provides the most flexibility for preprocessing your BAM files.</span> For example, with BamTable, you can trim the first several bases of the aligned reads to reduce the impact of post-mortem damage. 

We will first see how to run hapCon from samtools's pileup file. To generate the pileup file for SUA001, we need a bed file to specify regions of interest, which is in the dropbox link provided above. <span style='color:red'>Our bed file assumes that the contig name in your BAM file doesn't have chr or Chr prefix.</span> If that is the case for your BAM file, please reset the header of your BAM file by "samtools reheader -c 'perl -pe "s/^(@SQ.*)(\tSN:)chr/\$1\$2/"' in.bam > out.bam". If you are unsure about the contig name of your BAM file, you can check it by "samtools view -H in.bam".

We have assumed that you have put the BAM file at ./Data, and please change the path to bed file for 1240k panel according to your setup. After that, we can run the following to generate the pileup file. <span style='color:red'>Please make sure that the pileup file has suffix ".mpileup"</span>


In [3]:
path2bam="./Data/SUA001.bam"
path2bed1240k="/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/1240kChrX.bed" # Change this path according to your own setup
!samtools index $path2bam
!samtools mpileup --positions $path2bed1240k -r X -q 30 -Q 30 -o ./Data/SUA001.mpileup $path2bam

[mpileup] 1 samples in 1 input files


With the pileup file in hand, we can now ran hapCon to estimate contamination rate. Below is an example run with default setting. 

Please change the path to reference panel and meta data according to your setup. 

The function hapCon_chrom_BFGS should run for about 15s. It produces two output files, which by default reside in the same directory as the input mpileup file. The first output file is a hdf5 file, which is used as an intermediary data file for our method, and can be removed by setting cleanup=True in the function. The second file is the contamination estimate, which is named as $iid.hapCon.txt

In [1]:
from hapsburg.PackagesSupport.hapsburg_run import hapCon_chrom_BFGS
path2ref1240k="/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chrX.hdf5" # Change this path according to your own setup
path2meta="/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/meta_df_all.csv" # Change this path according to your own setup

In [4]:
!hapConX -m ./Data/SUA001.mpileup -r $path2ref1240k --meta $path2meta # run on pileup file

exclude 1033 sites outside the specified region
exclude 0 non-SNP sites
number of major reads at flanking sites: 10636
number of minor reads at flanking sites: 16
number of major reads at focal sites: 1291
number of minor reads at focal sites: 27
err rate at flanking sites: 0.001502
err rate at focal sites: 0.020486
saving sample as SUA001 in /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/SUA001.hdf5
estimated genotyping error by flanking sites: 0.001502
number of sites covered by at least one read: 3999, fraction covered: 0.085
hdf5 file saved to /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/SUA001.hdf5
finished reading mpileup file, takes 1.741.
number of sites covered by at least one read: 3999
hdf5 file saved to /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/SUA001.hdf5
estimated contamination rate: 0.102113(0.076802 - 0.127424)


Now you have finished a hapCon run on SUA001! The estimated contamination rate should be about 10%. This is a highly contaminated sample. Now let's try to run hapCon using output from BamTable. Here we are only using the most basic functionality of BamTable as an illustration. Please refer to [BamTable](https://bioinf.eva.mpg.de/BamTable/) for a full list of options available for BamTable. For example, restrict analysis to reads of certain lengths or trimming several bases from 5' or 3' end. We will use the same bed file for BamTable as we have done for samtools.

<span style='color:red'>Note: it is important to specify both -b and -A. -b means our input is bed file (0-indexed) and -A tells BamTable to use the fact that our BED file is sorted, which greatly speeds up computation. And please make sure that the ouput of BamTable ends with suffix ".BamTable"</span>

In [5]:
# Please change the path of BamTable executable according to your own setup
!/mnt/archgen/users/yilei/bin/BamTable-2.0.3/BamTable -f $path2bed1240k -b -A $path2bam > ./Data/SUA001.BamTable
!hapConX --bamtable ./Data/SUA001.BamTable -r $path2ref1240k --meta $path2meta # run on BAM file


exclude 1033 sites outside the specified region
exclude 0 non-SNP sites
number of major reads at flanking sites: 10636
number of minor reads at flanking sites: 16
number of major reads at focal sites: 1291
number of minor reads at focal sites: 27
err rate at flanking sites: 0.001502
err rate at focal sites: 0.020486
saving sample as SUA001 in /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/SUA001.hdf5
estimated genotyping error by flanking sites: 0.001502
number of sites covered by at least one read: 3999, fraction covered: 0.085
hdf5 file saved to /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/SUA001.hdf5
finished reading BamTable, takes 1.523
number of sites covered by at least one read: 3999
hdf5 file saved to /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/SUA001.hdf5
estimated contamination rate: 0.102113(0.076802 - 0.127424)


Finally let's try to run hapCon directly from a BAM file. Running hapCon come BAM file is a bit slower. The following code should take about 1min.

In [6]:
!hapConX --bam ./Data/SUA001.bam -r $path2ref1240k --meta $path2meta # run on BAM file

exclude 1033 sites outside the specified region
exclude 0 non-SNP sites
total number of mapped reads: 14755
number of major reads at flanking sites: 10521
number of minor reads at flanking sites: 15
number of major reads at focal sites: 1291
number of minor reads at focal sites: 27
err rate at flanking sites: 0.001424
err rate at focal sites: 0.020486
saving sample as SUA001 in /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/SUA001.hdf5
estimated genotyping error by flanking sites: 0.001424
number of sites covered by at least one read: 3999, fraction covered: 0.085
hdf5 file saved to /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/SUA001.hdf5
finished reading bam file, takes 61.997.
number of sites covered by at least one read: 3999
hdf5 file saved to /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/SUA001.hdf5
estimated contamination rate: 0.102190(0.076878 - 0.127501)


# hapCon with 1000G Reference Panel
With WGS data, we recommend using hapCon with the 1000G reference panel instead. This reference panel contains all biallelic sites with MAF greater than 5% in the 1000Genome dataset, therefore it is much more powerful than the 1240k reference panel. We will use DA43, a Mongolia XiongNu WGS sample. Let's first generate a pileup file for it.

We have assumed that you have put the BAM file of DA43 at ./Data, and please change the path to bed file for 1000G panel according to your setup. After that, we can run the following to generate the pileup file,

In [7]:
path2bam="./Data/DA43.bam"
path2bed1kg="/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/maf5FilterChrX.bed"
!samtools index $path2bam
!samtools mpileup --positions $path2bed1kg -r X -q 30 -Q 30 -o ./Data/DA43.mpileup $path2bam

[mpileup] 1 samples in 1 input files


With the pileup file, we can run hapCon on DA43 similar as we did to SUA001. Please change the path to the 1000G referene panel according to your setup. Running hapCon with 1000G panel is slower than that with 1240k panel, as it contains 4 times more sites. 

In [8]:
path2ref1kg="/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/maf5_filter_chrX.hdf5"
!hapConX -m ./Data/DA43.mpileup -r $path2ref1kg --meta $path2meta # run on pileup file

exclude 16544 sites outside the specified region
exclude 0 non-SNP sites
number of major reads at flanking sites: 16633
number of minor reads at flanking sites: 156
number of major reads at focal sites: 1977
number of minor reads at focal sites: 41
err rate at flanking sites: 0.009292
err rate at focal sites: 0.020317
saving sample as DA43 in /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/DA43.hdf5
estimated genotyping error by flanking sites: 0.009292
number of sites covered by at least one read: 17584, fraction covered: 0.085
hdf5 file saved to /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/DA43.hdf5
finished reading mpileup file, takes 20.630.
number of sites covered by at least one read: 17584
hdf5 file saved to /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/DA43.hdf5
estimated contamination rate: 0.034005(0.027509 - 0.040502)


The estimated contamination should be about 3%. Now you have finished your first trial with 1000G reference panel!

Alternatively, we can also run from BamTable.

In [9]:
# Please change the path of BamTable according to your own setup
!/mnt/archgen/users/yilei/bin/BamTable-2.0.3/BamTable -f $path2bed1kg -b -A $path2bam > ./Data/DA43.BamTable
!hapConX --bamtable ./Data/DA43.BamTable -r $path2ref1kg --meta $path2meta # run on BAM file


exclude 16544 sites outside the specified region
exclude 0 non-SNP sites
number of major reads at flanking sites: 16633
number of minor reads at flanking sites: 156
number of major reads at focal sites: 1977
number of minor reads at focal sites: 41
err rate at flanking sites: 0.009292
err rate at focal sites: 0.020317
saving sample as DA43 in /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/DA43.hdf5
estimated genotyping error by flanking sites: 0.009292
number of sites covered by at least one read: 17584, fraction covered: 0.085
hdf5 file saved to /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/DA43.hdf5
finished reading BamTable, takes 19.708
number of sites covered by at least one read: 17584
hdf5 file saved to /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/DA43.hdf5
estimated contamination rate: 0.034005(0.027509 - 0.040502)


Of course, we can also run directly from BAM file.

In [10]:
!hapConX --bam ./Data/DA43.bam -r $path2ref1kg --meta $path2meta # run on BAM file

exclude 16544 sites outside the specified region
exclude 0 non-SNP sites
total number of mapped reads: 281907
number of major reads at flanking sites: 16270
number of minor reads at flanking sites: 153
number of major reads at focal sites: 1977
number of minor reads at focal sites: 41
err rate at flanking sites: 0.009316
err rate at focal sites: 0.020317
saving sample as DA43 in /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/DA43.hdf5
estimated genotyping error by flanking sites: 0.009316
number of sites covered by at least one read: 17584, fraction covered: 0.085
hdf5 file saved to /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/DA43.hdf5
finished reading bam file, takes 239.403.
number of sites covered by at least one read: 17584
hdf5 file saved to /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/DA43.hdf5
estimated contamination rate: 0.033982(0.027485 - 0.040479)


# Running multiple samples in a batch

With the command line tool introduced above, it should be relatively easy to parallelize hapCon on multiple samples by submitting array jobs in a typical HPC server. If you prefer, however, there is also a script [hapCONX_batch.py](https://github.com/hyl317/hapROH/blob/master/bam/hapCONX_batch.py) with which you can run multiple samples in a batch without submitting array jobs. In the end a .tsv file will be made summarizing the results for each sample. For this, you need to provide a file that contains a list of pileup/BamTable/BAM files. Each line in the file should have two columns, the first column is the sample IID, and the second column is the path to pileup/BAM files. The two columns should be separted by a tab.

In the final .tsv file, each row summarizes the result for one sample. The first column is the sample IID, the second column is of the form "MLE for contamination(low 95% CI - high 95% CI)", and the third column is the number of sites covered by at least one read. The third column essentailly says how many sites are utilized in contamination estimates (i.e, the length of the HMM chain). A rule of thumb is that we need at least 2000 sites to have confident contamination estimates. 


# what's more
You are welcome to visit our [readthedocs](https://haproh.readthedocs.io/en/latest/autoapi/hapsburg/PackagesSupport/hapsburg_run/index.html#hapsburg.PackagesSupport.hapsburg_run.hapCon_chrom_BFGS) site to see new updates.